In [57]:
@njit
def getOrientedAction(state,ratio,rotation,grad):
     #TODO *ratio
    #t=1 if state=0 and t = ratio if state = 1
    # print(np.dot(rotation,r0.T).T,cGrad)
    # print(np.dot(np.dot(rotation,r0.T).T,cGrad))

    grad = np.array([0., 0., grad])
    ratios = ratio + np.dot(np.dot(rotation,r0[1:].T).T,grad)

    
    t1,t2,t3,t4 = np.where(state==0,np.ones(4),ratios)
    #print(t1,t2,t3,t4)
    
    sum = t1*t2*t3 + t1*t2*t4 + t1*t3*t4 + t2*t3*t4
    #probs = np.array([t2*t3*t4/sum,t1*t3*t4/sum,t1*t2*t4/sum,t2*t3*t1/sum])
    randomNumber = np.random.uniform(0,1,1)
    if randomNumber <= t4*t2*t3/sum:
        changingJoint = 0
     #   deltat = getDeltat(t1,t2,t3,t4,t1)
    elif(randomNumber <= (t4*t2*t1+t4*t2*t3)/sum):  
        changingJoint = 1
#        deltat = getDeltat(t1,t2,t3,t4,t2)
    elif randomNumber <= (t4*t2*t1+t4*t2*t3+t4*t2*t1)/sum:
        changingJoint = 2
        # deltat = getDeltat(t1,t2,t3,t4,t3)
    else:
        changingJoint = 3
        # deltat = getDeltat(t1,t2,t3,t4,t4)

    #changingJoint = np.random.choice(np.arange(4),p=probs)
    # action = np.concatenate((state,changingJoint.reshape((1,))),axis=0)
    # deltat = 1 if state[changingJoint]==0 else ratio
    return changingJoint,getDeltat2(t1,t2,t3,t4)

@njit
def doOrientedStep(Data,state,ratio,rotation,grad):
    action,deltat = getOrientedAction(state,ratio,rotation,grad)
    new_state = state.copy()
    new_state[action] = 1 - new_state[action]
    delta = Data[fromBinary(state)*4 + action][0,:]
    rotation = Data[fromBinary(state)*4 + action][1:,:]
    return new_state,delta,rotation,deltat

In [58]:
@njit
def Orientedperturbe(Data,perturbingSteps,ratio,grad):
    time = 0
    pertrubingPose = np.array([0.,0.,0.])
    perturbingRotation = np.eye(3)
    state = np.ones(4,dtype=np.int64)
    pertrubingPoses = np.zeros((perturbingSteps,3))
    pertrubingTimes = np.zeros(perturbingSteps)

    for t in range(perturbingSteps):
        newState,delta,rotation,deltat = doOrientedStep(Data,state,ratio,perturbingRotation,grad)
        state = newState
        pertrubingPose += mul2(perturbingRotation,delta)
        perturbingRotation = mul(rotation,perturbingRotation)
        pertrubingPoses[t] = pertrubingPose
        time += deltat
        pertrubingTimes[t] = time

    return pertrubingPoses,pertrubingTimes

In [120]:
@njit
def getOrientedPerturbingDiffusionCoefForRatio(Data,ratio,grad):
    iterations = 10_00
    perturbingSteps = 10_000
    meanFinalPoses = np.zeros((perturbingSteps))
    meanFinalTimes = np.zeros(perturbingSteps)
    for iteration in range(iterations):
        pertrubingPoses,pertrubingTimes = Orientedperturbe(Data,perturbingSteps,ratio,grad)
        #print(np.sum(np.square(pertrubingPoses[:]),axis=1).shape,meanFinalPoses.shape)
        meanFinalPoses += pertrubingPoses[:,2]
        meanFinalTimes += pertrubingTimes
        #print(iteration)
     
    # plt.plot(meanFinalTimes,meanFinalPoses)
    # plt.show()
    
    # return slope  #*some Coef
    return meanFinalTimes/iterations,-meanFinalPoses/iterations

def getOrientedPerturbingDiffusionCoef(Data,ratio,grads):
    diffs =np.zeros(grads.shape)
    for i in range(grads.shape[0]):
        print(i)
        meanFinalTimes,meanFinalPoses= getOrientedPerturbingDiffusionCoefForRatio(Data,ratio,grads[i])
        slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(meanFinalTimes,meanFinalPoses)
        diffs[i] =  slope 
    return diffs


def getOrientedPerturbingDiffusionCoef2(Data,ratios,grad):
    diffs =np.zeros(grads.shape)
    for i in range(ratios.shape[0]):
        print(i)
        meanFinalTimes,meanFinalPoses= getOrientedPerturbingDiffusionCoefForRatio(Data,ratios[i],grad*ratios[i])
        slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(meanFinalTimes,meanFinalPoses)
        diffs[i] =  slope 
    return diffs